In [1]:
using Pkg

Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("EAGO")
Pkg.add("HiGHS")
Pkg.add("BenchmarkTools")

ENV["CPLEX_STUDIO_BINARIES"] = "/opt/ibm/ILOG/CPLEX_Studio_Community221/cplex/bin/x86-64_linux"
Pkg.add("CPLEX")
Pkg.build("CPLEX")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
    Building CPLEX → `~/.julia/scratchsp

In [2]:
struct Delivery
    x::Float64
    y::Float64
end;
function random_delivery()
    x = rand(0.0:0.1:100.0)
    y = rand(0.0:0.1:100.0)
    return Delivery(x, y)
end;

struct TSProblem
    deliveries::Array{Delivery}
end;

function random_instance(n_deliveries)
    deliveries = [random_delivery() for _=1:n_deliveries]
    problem = TSProblem(deliveries)
    return problem
end;

function dist(del1::Delivery, del2::Delivery)
    return sqrt((del1.x - del2.x)^2 + (del1.y - del2.y)^2)
end
function calc_travelmatrix(deliveries::Array{Delivery})
    tm = zeros(Float64, length(deliveries), length(deliveries))
    for i = 1:length(deliveries)
        for j = 1:length(deliveries)
            tm[i, j] = dist(deliveries[i], deliveries[j])
        end
    end
    return tm
end


calc_travelmatrix (generic function with 1 method)

In [3]:
using JuMP

import GLPK
import EAGO
import HiGHS
import CPLEX

using BenchmarkTools

function prep_model(deliveries::Int64, solver)
    problem=random_instance(deliveries)
    travelmatrix = calc_travelmatrix(problem.deliveries)
    model = Model(solver)
    # route is an adjence matrix representing a route traveled
    route=@variable(model, route[1:length(problem.deliveries), 1:length(problem.deliveries)], Bin)
    # mtzu is a helper variable to ensure no subtours are allowed (only one continous tour)
    # see MTZ constraint
    mtzu = @variable(model, mtzu[1:length(problem.deliveries)], Int)

    # ensure all events are planned
    @constraint(model, [i = 1:length(problem.deliveries)], sum(route[i, :]) == 1.0)
    # ensure there is just one route
    @constraint(model, [c = 1:length(problem.deliveries)], sum(route[:, c]) == 1.0)
    # disallow traveling to itself
    @constraint(model, [j = 1:length(problem.deliveries)], route[j, j] == 0)

    # MTZ constraints for removing subtours
    n = length(problem.deliveries)
    @constraint(model, [ui = 1:n, uj = 2:n], mtzu[ui] + route[ui, uj] <= mtzu[uj]+ (n - 1) * (1 - route[ui, uj]) )

    traveltime = travelmatrix.* route 
    @objective(model, Min, sum(traveltime))
    return model
end

model = prep_model(5, GLPK.Optimizer)
optimize!(model)

model = prep_model(5, EAGO.Optimizer)
optimize!(model)

model = prep_model(5, HiGHS.Optimizer)
set_optimizer_attribute(model, "log_to_console", false)
optimize!(model)

model = prep_model(5, CPLEX.Optimizer)
set_optimizer_attribute(model, "CPXPARAM_ScreenOutput", 0)
optimize!(model)



In [4]:
model = prep_model(5, GLPK.Optimizer)
@time  optimize!(model)

  0.001040 seconds (922 allocations: 53.016 KiB)


In [5]:
model = prep_model(10, GLPK.Optimizer)
@time  optimize!(model)

  0.062086 seconds (2.65 k allocations: 157.672 KiB)


In [6]:
@benchmark optimize!(m) setup=(m=prep_model(10, GLPK.Optimizer)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):   5.897 ms … 89.283 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.162 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.556 ms ± 22.390 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █  ▃     ▃    ▃                                              
  █▁▁█▇▁▁▁▇█▇▁▇▁█▁▇▁▇▁▁▁▁▁▇▁▁▁▁▁▁▇▁▁▁▇▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  5.9 ms          Histogram: frequency by time        89.3 ms <

 Memory estimate: 60.37 KiB, allocs estimate: 1153.

In [7]:
@benchmark optimize!(m) setup=(m=prep_model(10, EAGO.Optimizer)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):   53.411 ms … 913.161 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     204.268 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   326.879 ms ± 265.734 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃     ▃   █                                                    
  █▁▁▁▁▇█▇▇▇█▇▇▁▁▁▁▁▁▁▁▁▇▁▇▁▁▁▁▁▁▁▁▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▇▁▁▁▁▇ ▁
  53.4 ms          Histogram: frequency by time          913 ms <

 Memory estimate: 553.16 KiB, allocs estimate: 10947.

In [8]:
function highs_prep(deliveries::Int64)
    m = prep_model(deliveries, HiGHS.Optimizer)
    set_optimizer_attribute(m, "log_to_console", false)
    return m
end

@benchmark optimize!(m) setup=(m=highs_prep(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):    5.098 ms …    2.185 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     110.645 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   341.199 ms ± 545.732 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▁▄▅▁▁▄▁▄▁▄▁▄▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  5.1 ms           Histogram: frequency by time          2.19 s <

 Memory estimate: 93.65 KiB, allocs estimate: 712.

In [9]:
function cplex_prep(deliveries::Int64)
    m = prep_model(deliveries, CPLEX.Optimizer)
    set_optimizer_attribute(m, "CPXPARAM_ScreenOutput", 0)
    return m
end

@benchmark optimize!(m) setup=(m=cplex_prep(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):  3.964 ms … 28.454 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.378 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.751 ms ±  5.487 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁█ ▁█ █▁█ ▁▁  ▁  ▁   ▁ ▁                                ▁  
  ███▁██▁███▁██▁▁█▁▁█▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.96 ms        Histogram: frequency by time        28.5 ms <

 Memory estimate: 76.17 KiB, allocs estimate: 1663.

In [10]:
model=cplex_prep(30)
@time  optimize!(model)

  0.399530 seconds (40.40 k allocations: 1.920 MiB)
